## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
attrition_df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [5]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Age', 'DistanceFromHome','Education', 'EducationField',
             'HourlyRate','JobInvolvement', 'JobLevel', 'JobRole',
             'JobSatisfaction', 'NumCompaniesWorked', 'PercentSalaryHike','PerformanceRating'
             ]

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


Age                    int64
DistanceFromHome       int64
Education              int64
EducationField        object
HourlyRate             int64
JobInvolvement         int64
JobLevel               int64
JobRole               object
JobSatisfaction        int64
NumCompaniesWorked     int64
PercentSalaryHike      int64
PerformanceRating      int64
dtype: object

In [6]:
print(X_df['EducationField'].unique())
print(X_df['JobRole'].unique())

['Life Sciences' 'Other' 'Medical' 'Marketing' 'Technical Degree'
 'Human Resources']
['Sales Executive' 'Research Scientist' 'Laboratory Technician'
 'Manufacturing Director' 'Healthcare Representative' 'Manager'
 'Sales Representative' 'Research Director' 'Human Resources']


In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
columns_encoded = encoder.fit_transform(X_df[['EducationField', 'JobRole']])
column_names = encoder.get_feature_names_out(['EducationField', 'JobRole'])
columns_encoded_df = pd.DataFrame(columns_encoded, columns=column_names)
X_df_encoded = pd.concat([X_df.drop(['EducationField', 'JobRole'], axis=1), columns_encoded_df], axis=1)
X_df_encoded.dtypes


Age                                    int64
DistanceFromHome                       int64
Education                              int64
HourlyRate                             int64
JobInvolvement                         int64
JobLevel                               int64
JobSatisfaction                        int64
NumCompaniesWorked                     int64
PercentSalaryHike                      int64
PerformanceRating                      int64
EducationField_Human Resources       float64
EducationField_Life Sciences         float64
EducationField_Marketing             float64
EducationField_Medical               float64
EducationField_Other                 float64
EducationField_Technical Degree      float64
JobRole_Healthcare Representative    float64
JobRole_Human Resources              float64
JobRole_Laboratory Technician        float64
JobRole_Manager                      float64
JobRole_Manufacturing Director       float64
JobRole_Research Director            float64
JobRole_Re

In [8]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df_encoded, y_df)


In [9]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [10]:
print(y_train['Department'].unique())
print(y_train['Attrition'].unique())

['Research & Development' 'Sales' 'Human Resources']
['No' 'Yes']


In [11]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_variables_train = department_encoder.transform(y_train[['Department']])
department_variables_test = department_encoder.transform(y_test[['Department']])
department_variables_train



array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [12]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_variables_train = attrition_encoder.transform(y_train[['Attrition']])
attrition_variables_test = attrition_encoder.transform(y_test[['Attrition']])
attrition_variables_train


array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

## Create, Compile, and Train the Model

In [13]:
# Find the number of columns in the X training data
X_shape = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(X_shape,), name = 'input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [14]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)

In [18]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(1, activation='softmax', name='attrition_output')(attrition_hidden)


In [19]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 25)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      1,664 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      1,056 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 32)        │      1,056 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ dense_2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 1)         │         33 │ dense_4[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,988 (23.39 KB)

 Trainable params: 5,988 (23.39 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Train the model
model.fit(
    X_train_scaled,
    {'department_output': department_variables_train, 'attrition_output': attrition_variables_train},
    epochs=30,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.1642 - attrition_output_loss: 0.3280 - department_output_accuracy: 0.9945 - department_output_loss: 0.0195 - loss: 0.3474 - val_attrition_output_accuracy: 0.1131 - val_attrition_output_loss: 0.3657 - val_department_output_accuracy: 0.9774 - val_department_output_loss: 0.1092 - val_loss: 0.4752
Epoch 2/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.1714 - attrition_output_loss: 0.3433 - department_output_accuracy: 0.9877 - department_output_loss: 0.0214 - loss: 0.3646 - val_attrition_output_accuracy: 0.1131 - val_attrition_output_loss: 0.3549 - val_department_output_accuracy: 0.9774 - val_department_output_loss: 0.1204 - val_loss: 0.4753
Epoch 3/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.1781 - attrition_output_loss: 0.3140 - department_output_accuracy: 0.9922 - department_output_loss: 0.0174 - loss: 0.3314 - val_attrition_output_accuracy: 0.1131 - val

In [25]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output': department_variables_test, 'attrition_output': attrition_variables_test})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.1638 - attrition_output_loss: 0.7191 - department_output_accuracy: 0.9735 - department_output_loss: 0.1506 - loss: 0.8674     


[0.855970025062561,
 0.17225462198257446,
 0.6978240609169006,
 0.14945651590824127,
 0.970108687877655]

In [24]:
# Print the accuracy for both department and attrition
test_results[0]
print('Department predictions accuracy: ', test_results[4])
print('Attrition predictions accuracy: ', )

0.855970025062561

Department predictions accuracy: 0.5271739363670349
Attrition predictions accuracy: 0.826086938381195

In [30]:
attrition_df['Attrition'].value_counts()

Attrition
No     1233
Yes     237
Name: count, dtype: int64

In [31]:
attrition_df['Department'].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is not the best measure for attrition or department because the dataset is very unbalanced. There are a lot more 'No's in the attrition column and there are unequal numbers of people in each department. Therefore the accuracy would go up even if the model does a poor job of predicting the less common categories. 
2. I used Softmax on the Department data because each department is mututally exclusive, so they should predicatably add to 1. I used Softmax for attrition because it is simply a one binary variable and therefore must be mutually exclusive. 
3. It could be improved by:
    * Adding more columns from the original data
    * using a better measure than accuracy for scoring the model
    * balancing the data by using oversampling or undersampling techniques
    * adding more layers
    * adding more neurons
    * running it for more epochs